In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
kgs.preallocate_matrices=False
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
#kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
#kgs.cache_dir_read = kgs.cache_dir_write

F:/seismic/models/brendan/*.pth
Loading:  F:/seismic/models/brendan\unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
model.read_cache = False

styleB not active yet


In [3]:
%%time
data = kgs.load_all_train_data(validation_only=True)[::10]
model.models[1].run_in_parallel = True
data_out = model.infer(data)

Processing in parallel : 100%|███████████████████████████████████████████████████████| 500/500 [00:34<00:00, 14.59it/s]


CPU times: total: 2.88 s
Wall time: 39.4 s


In [24]:
dfTimings = pd.read_csv(kgs.timing_filename)
dfTimings = dfTimings.groupby('family')['time_taken'].mean().to_frame('time_taken')
#dfTimings.loc['FlatFault_B'].to_numpy()[0]

In [25]:
#kgs.disable_caching = True
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    res['time_taken'].append(dfTimings.loc[f].to_numpy()[0])
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

Importing cache brendan: 100%|███████████████████████████████████████████████████████| 50/50 [00:00<00:00, 4166.72it/s]



Brendan local score:  23.5210680861368
My local score:  19.351145971779335
Estimate public LB:  23.694204784676526


,family,score_brendan,score_mine,score_mine_max,time_taken
0,CurveFault_A,3.931313,3.931313,18.090105,0.166579
1,CurveFault_B,67.427065,67.427065,143.542449,0.008927
2,CurveVel_A,8.550278,8.550278,21.201429,0.008863
3,CurveVel_B,34.534996,34.534996,174.873469,0.009391
4,FlatFault_A,2.246222,2.246222,7.928230,0.009710
5,FlatFault_B,23.314437,23.314437,105.244286,0.009719
6,FlatVel_A,1.159649,0.048000,0.228571,0.118717
7,FlatVel_B,7.614163,0.021143,0.328571,0.121904
8,Style_A,38.102926,5.108374,10.033272,0.087295
9,Style_B,48.329632,48.329632,68.906210,0.033624


In [5]:
# No extra model: 28.8
# FlatVel:
# StyleA: 

In [6]:
28-30*0.081

25.57